In [1]:
import pandas as pd
import numpy as np


In [2]:
# data from Ken French's web site:
# http://mba.tuck.dartmouth.edu/pages/faculty/ken.french/data_library.html
# data set: Industry Portfolios, 10 Industry Portfolios 
fileName = 'data/10_Industry_Portfolios_Monthly.csv' # change as needed


In [3]:
df = pd.read_csv(fileName) # read data from csv file
df = df.dropna() # remove missing observations
T, N = df.shape # T is the number of periods
N = N - 1 # number of risky assets ignoring date
print('Data for %d assests over %d periods' %(N, T)) # shape of data frame
df.head() # display first 5 rows


Data for 10 assests over 1098 periods


,Date,NoDur,Durbl,Manuf,Enrgy,HiTec,Telcm,Shops,Hlth,Utils,Other
0,192607,1.45,15.55,4.69,-1.18,2.90,0.83,0.11,1.77,7.04,2.16
1,192608,3.97,3.68,2.81,3.47,2.66,2.17,-0.71,4.25,-1.69,4.38
2,192609,1.14,4.80,1.15,-3.39,-0.38,2.41,0.21,0.69,2.04,0.29
3,192610,-1.24,-8.23,-3.63,-0.78,-4.58,-0.11,-2.29,-0.57,-2.63,-2.85
4,192611,5.21,-0.19,4.10,0.01,4.71,1.63,6.43,5.42,3.71,2.11


In [4]:
def getData(df, start, M):
    """returns excess returns for the N assets over a window of M periods from start"""
    return df.loc[start:(start+M-1),df.columns != 'Date'].astype(float).as_matrix()


In [5]:
def naiveStrategy(N):
    """returns equal weights 1/N for N risky assets"""
    return np.ones(N)/N


In [6]:
def optimalWeights(m, c):
    """returns optimal normalized weights (equation 3 in DeMiguel)
    given mean vector m and covariance matrix c for N risky assets"""
    covI = np.linalg.inv(c) # inverse of covariance
    w = np.matmul(covI, m) # unnormalized optimal weights
    return w/sum(w) # normalized optimal weights


In [7]:
def meanVarianceStrategy(x):
    """returns optimal normalized weights for N risky assets 
    using Markowitz (1952) mean-variance strategy
    based on sample mean and covariance matrix of observations x"""
    m = np.mean(x,axis=0) # mean vector
    c = np.cov(x, rowvar=False)
    return optimalWeights(m, c)
    

In [8]:
def returns(df, t, w):
    """Given weight vector w, computes returns in period t"""
    x = getData(df, t, 1)
    return np.matmul(x,w)[0]


In [9]:
def SharpeRatio(returns):
    """returns Sharpe ratio given returns""" 
    m, s = np.mean(returns), np.std(returns)
    return m/s
    

In [10]:
def evaluateNaiveStrategy(M):
    w = naiveStrategy(N) # fixed weights
    res = []
    for t in range(T-M-2):
        x = getData(df, t, M)
        r = returns(df, t+M+1, w)
        res.append(r)
    SR = SharpeRatio(res)
    print('Sharpe Ratio for Naive Strategy = %4.4f' %SR)
    return SR
    

In [11]:
def evaluateMeanVarianceStrategy(M):
    resInSample, resOutOfSample = [], []
    for t in range(T-M-2):
        x = getData(df, t, M)
        w = meanVarianceStrategy(x)
        ri = returns(df, t+M, w)
        ro = returns(df, t+M+1, w)
        resInSample.append(ri)
        resOutOfSample.append(ro)
    SRI, SRO = SharpeRatio(resInSample), SharpeRatio(resOutOfSample)
    print('Sharpe Ratio for Mean Variance Strategy: Insample = %4.4f, OutOfsample = %4.4f' %(SRI, SRO))
    return SRI, SRO


In [12]:
for M in range(60,601,60):
    print('Window size = %d' %M)
    SRNS = evaluateNaiveStrategy(M)
    SRI, SRO = evaluateMeanVarianceStrategy(M)
    print()
    

Window size = 60
Sharpe Ratio for Naive Strategy = 0.2001
Sharpe Ratio for Mean Variance Strategy: Insample = 0.0438, OutOfsample = 0.0654

Window size = 120
Sharpe Ratio for Naive Strategy = 0.2320
Sharpe Ratio for Mean Variance Strategy: Insample = 0.1049, OutOfsample = 0.0987

Window size = 180
Sharpe Ratio for Naive Strategy = 0.2615
Sharpe Ratio for Mean Variance Strategy: Insample = 0.1875, OutOfsample = 0.1905

Window size = 240
Sharpe Ratio for Naive Strategy = 0.2510
Sharpe Ratio for Mean Variance Strategy: Insample = 0.2366, OutOfsample = 0.2345

Window size = 300
Sharpe Ratio for Naive Strategy = 0.2541
Sharpe Ratio for Mean Variance Strategy: Insample = 0.2477, OutOfsample = 0.2403

Window size = 360
Sharpe Ratio for Naive Strategy = 0.2402
Sharpe Ratio for Mean Variance Strategy: Insample = 0.2407, OutOfsample = 0.2371

Window size = 420
Sharpe Ratio for Naive Strategy = 0.2301
Sharpe Ratio for Mean Variance Strategy: Insample = 0.2226, OutOfsample = 0.2179

Window size = 